<a href="https://colab.research.google.com/github/jiobu1/DS-Unit-2-Applied-Modeling/blob/master/Jisha_Obukwelu_DSPT3_231_Applied_Modeling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency > 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from google.colab import files
upload = files.upload()

Saving states_all.csv to states_all.csv


###***Choose your target. Which column in your tabular dataset will you predict?***

This project will predict state's passing rate on the naep assessment based on features from the dataset. 

###***Is your problem regression or classification?***

This is a classification problem, I will be engineering the classification column based on the cut scores (score range) from the NAEP assessment for basic, proficient, and advanced.

###***How is your target distributed?***
Classification: How many classes? Are the classes imbalanced?

In [0]:
import pandas as pd
import numpy as np

In [0]:
all_states = pd.read_csv('states_all.csv')

In [12]:
all_states.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,NaN,174053.0,8224.0,55460.0,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,NaN,37451.0,2371.0,10152.0,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,NaN,609114.0,2544.0,53497.0,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,NaN,145212.0,808.0,33511.0,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,NaN,2044688.0,59067.0,431763.0,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


There should be three classes (that I will engineer) but the classes are gravely imbalanced with one class not having any values in it. 

###***Choose your evaluation metric(s).***
Classification: Is your majority class frequency > 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?

####4TH GRADE READING

In [31]:
all_states['AVG_READING_4_SCORE'] = all_states['AVG_READING_4_SCORE'].fillna(0)
all_states['AVG_READING_4_SCORE'] = all_states['AVG_READING_4_SCORE'].astype(int)
all_states['AVG_READING_4_SCORE']

0       207
1         0
2       206
3       208
4       196
       ... 
1487    227
1488    223
1489    217
1490    219
1491    226
Name: AVG_READING_4_SCORE, Length: 1492, dtype: int64

In [34]:
def proficiency(row):
    if row['AVG_READING_4_SCORE'] < 238 or row['AVG_READING_4_SCORE'] > 0:
        val = 'basic'
    elif row['AVG_READING_4_SCORE'] < 268 or row['AVG_READING_4_SCORE'] >= 238:
        val = 'proficient'
    elif row['AVG_READING_4_SCORE'] >= 268:
        val = 'advanced'
    else:
        val = 0
    return val

all_states['READING_PROF_4'] = all_states.apply(proficiency, axis=1)
all_states['READING_PROF_4']

0       basic
1       basic
2       basic
3       basic
4       basic
        ...  
1487    basic
1488    basic
1489    basic
1490    basic
1491    basic
Name: READING_PROF_4, Length: 1492, dtype: object

In [35]:
min(all_states['AVG_READING_4_SCORE']), max(all_states['AVG_READING_4_SCORE'])

(0, 236)

So this won't work - It's is 100% basic. Let me see if I can use another one of the score columns for predictions.




####8TH GRADE READING

In [36]:
all_states['AVG_READING_8_SCORE'] = all_states['AVG_READING_8_SCORE'].fillna(0)
all_states['AVG_READING_8_SCORE'] = all_states['AVG_READING_8_SCORE'].astype(int)
all_states['AVG_READING_8_SCORE']

0         0
1       258
2       262
3       264
4         0
       ... 
1487    273
1488    267
1489    271
1490    258
1491    269
Name: AVG_READING_8_SCORE, Length: 1492, dtype: int64

In [38]:
def proficiency(row):
    if row['AVG_READING_8_SCORE'] < 281 or row['AVG_READING_8_SCORE'] > 0:
        val = 'basic'
    elif row['AVG_READING_8_SCORE'] < 323 or row['AVG_READING_8_SCORE'] >= 281:
        val = 'proficient'
    elif row['AVG_READING_8_SCORE'] >= 323:
        val = 'advanced'
    else:
        val = 0
    return val

all_states['READING_PROF_8'] = all_states.apply(proficiency, axis=1)
all_states['READING_PROF_8']

0       basic
1       basic
2       basic
3       basic
4       basic
        ...  
1487    basic
1488    basic
1489    basic
1490    basic
1491    basic
Name: READING_PROF_8, Length: 1492, dtype: object

In [39]:
min(all_states['AVG_READING_8_SCORE']), max(all_states['AVG_READING_8_SCORE'])

(0, 280)

So this won't work either - It's is 100% basic. Let me see if I can use another one of the score columns for predictions.


####4TH GRADE MATH 

In [54]:
min(all_states['AVG_MATH_4_SCORE']), max(all_states['AVG_MATH_4_SCORE'])

(0, 253)

In [53]:
all_states['AVG_MATH_4_SCORE'] = all_states['AVG_MATH_4_SCORE'].fillna(0)
all_states['AVG_MATH_4_SCORE'] = all_states['AVG_MATH_4_SCORE'].astype(int)
all_states['AVG_MATH_4_SCORE']

0       208
1         0
2       215
3       210
4       208
       ... 
1487    248
1488    241
1489    236
1490    240
1491    247
Name: AVG_MATH_4_SCORE, Length: 1492, dtype: int64

In [55]:
def proficiency(row):
    if row['AVG_MATH_4_SCORE'] < 249 or row['AVG_MATH_4_SCORE'] > 0:
        val = 'basic'
    elif row['AVG_MATH_4_SCORE'] < 282 or row['AVG_MATH_4_SCORE'] >= 249:
        val = 'proficient'
    elif row['AVG_MATH_4_SCORE'] >= 282:
        val = 'advanced'
    else:
        val = 0
    return val

all_states['MATH_PROF_4'] = all_states.apply(proficiency, axis=1)
all_states['MATH_PROF_4']

0       basic
1       basic
2       basic
3       basic
4       basic
        ...  
1487    basic
1488    basic
1489    basic
1490    basic
1491    basic
Name: MATH_PROF_4, Length: 1492, dtype: object

In [56]:
all_states['MATH_PROF_4'].value_counts()

basic    1492
Name: MATH_PROF_4, dtype: int64

In [58]:
all_states['AVG_MATH_4_SCORE'].value_counts()

0      956
240     35
243     30
237     29
241     29
242     25
238     24
235     23
236     23
239     21
230     19
231     18
244     18
245     17
246     16
232     16
234     14
229     14
228     12
227     11
226     11
233     10
215     10
224      9
247      8
217      8
213      7
220      7
219      7
225      7
222      6
248      6
249      5
223      5
221      4
210      3
208      3
218      3
252      3
253      3
214      2
212      2
211      2
251      2
209      2
204      2
250      1
201      1
193      1
192      1
187      1
Name: AVG_MATH_4_SCORE, dtype: int64

This would not work either but I am wondering why my function did not pick up on the 6 proficient score?

####8TH GRADE MATH

In [46]:
min(all_states['AVG_MATH_8_SCORE']), max(all_states['AVG_MATH_8_SCORE'])

(0, 300)

In [47]:
all_states['AVG_MATH_8_SCORE'] = all_states['AVG_MATH_8_SCORE'].fillna(0)
all_states['AVG_MATH_8_SCORE'] = all_states['AVG_MATH_8_SCORE'].astype(int)
all_states['AVG_MATH_8_SCORE']

0       252
1         0
2       265
3       256
4       260
       ... 
1487    290
1488    289
1489    273
1490    288
1491    288
Name: AVG_MATH_8_SCORE, Length: 1492, dtype: int64

In [48]:
def proficiency(row):
    if row['AVG_MATH_8_SCORE'] < 299 or row['AVG_MATH_8_SCORE'] > 0:
        val = 'basic'
    elif row['AVG_MATH_8_SCORE'] < 333 or row['AVG_MATH_8_SCORE'] >= 299:
        val = 'proficient'
    elif row['AVG_MATH_8_SCORE'] >= 333:
        val = 'advanced'
    else:
        val = 0
    return val

all_states['MATH_PROF_8'] = all_states.apply(proficiency, axis=1)
all_states['MATH_PROF_8']

0       basic
1       basic
2       basic
3       basic
4       basic
        ...  
1487    basic
1488    basic
1489    basic
1490    basic
1491    basic
Name: MATH_PROF_8, Length: 1492, dtype: object

In [49]:
all_states['MATH_PROF_8'].value_counts()

basic    1492
Name: MATH_PROF_8, dtype: int64

In [50]:
all_states['AVG_MATH_8_SCORE'].value_counts()

0      960
283     36
286     30
281     29
277     26
280     23
284     23
282     22
288     22
285     21
287     19
279     19
275     17
274     16
278     16
270     15
276     13
271     13
273     11
268     11
289     10
269     10
272     10
262     10
266      9
290      9
265      8
291      8
267      7
294      6
263      6
292      6
260      5
259      5
264      4
261      4
293      4
258      3
256      2
252      2
234      2
253      2
295      2
296      2
297      2
298      2
257      1
254      1
250      1
249      1
248      1
246      1
245      1
243      1
232      1
300      1
Name: AVG_MATH_8_SCORE, dtype: int64

This would not work either but I am wondering why my function did not pick up on the lone proficient score?



###***Choose which observations you will use to train, validate, and test your model.***

I would split this dataset by date. That would make the most sense in this context. 

Does not make sense to split since as a classification problem, greater than 70% of the majority class for any of the predictive features is the majority class of basic. 

###***Begin to clean and explore your data.***

For this dataset, I would delete the years where the NAEP test scores are not available since that does not give anything for me to train/validate/test the data on. 

###***Begin to choose which features, if any, to exclude. Would some features "leak" future information?***

If I were to use this dataset, I would delete the scores since that would definitely delete information. 
